<a href="https://colab.research.google.com/github/INT-Group-9/ColabNotebook/blob/main/cifa10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Test